In [25]:
import ollama
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Manip des données et transformation

In [26]:
os.getcwd()

'/home/onyxia/work/Ollama'

In [82]:
df=pd.read_csv('/home/onyxia/work/Ollama/data/1.csv')
df = df.dropna(subset=['sentiment'])
df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})
df=df[['Contenu','sentiment']]
df['Contenu'] = df['Contenu'].str.replace("\n", " ")
df_train=df
df.head()

/tmp/ipykernel_3651/1991247669.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})


,Contenu,sentiment
0,Accueil Voyage Salon Nautique de Paris 2019: p...,0
1,"Après Commercy et Saint-Nazaire, Montceau-les-...",0
2,Les conducteurs de locomotives revendiquent un...,1
3,Accueil Flash Eco Le Covid-19 et ses restricti...,0
4,Saintes : grève des psychologues au centre hos...,1


In [31]:
df_motiflast = pd.read_csv('/home/onyxia/work/Ollama/data/motiflast.csv')
df=df_motiflast[['text',"sentiment"]]

In [65]:
df_train, df_test = train_test_split(df, test_size=0.95, random_state=0)
df_train.head()

,Contenu,sentiment
152,Une Gilet jaune finit au tribunal pour avoir «...,0
290,Accueil Politique Guadeloupe : Fillon veut cro...,0
518,Accueil Flash Actu IDF: 90 km de ralentissemen...,0
182,Accueil Flash Eco Royaume-Uni : les ambulancie...,1
527,Accueil Économie Conjoncture Réservé aux abonn...,0


## Experimentation chat

In [57]:
content=df_train["Contenu"][152]

In [58]:
content

"Une Gilet jaune finit au tribunal pour avoir «gazé» un commissaire L’opposition entre les Gilets jaunes et les policiers, lundi soir devant le Louvre-Lens a quelque peu dégénéré. Jusqu’à ce qu’une Gilet jaune originaire de La Bassée insulte un commissaire et l’asperge de gaz poivré. Ce qui lui a valu un défèrement au tribunal de Béthune. Article réservé aux abonnés  Cet article est réservé aux abonnés Abonnez-vous pour être le mieux informé de votre quartier S'abonner Ou directement avec votre compte Google  Annulable à tout moment Mes avantages : L'intégralité des articles gratuits et abonnés Votre journal numérique chaque jour Les privilèges du Club Abonnés Déjà un compte ? Se connecter"

In [75]:
instruction_1="""Je veux t'envoyer un article et tu vas faire une classification binaire. Je veux que tu dises, Oui ou non selon qu il concernent une greve ou pas. Je veux que la sortie sois OUI ou NON
      La greve doit etre considérée dans ce sens : La grève est la dénomination donnée à un mouvement collectif pris à l'initiative de tout ou partie du personnel d'une entreprise, destiné en général à contraindre 
      l'employeur à la négociation des conditions de travail et de rémunération. La grève a cependant changé d'orientation et de modalité car, outre la protestation dont elle se veut être l'expression, elle peut aussi 
      être décidée dans un but de solidarité avec d'autres entreprises même si les protestataires n'appartiennent pas au même employeur, pour exprimer la crainte d'une décision future ou dans un but politique. 
      Elle est exercée tant par le personnel des entreprises privées que par les agents des services publics.
      En cas de doute je veux que tu repondes OUI. Pareil si tu n'a pas de moyen pour classifier la requete que je t'envoie, reponds simplement OUI. 
      Si l'article concerne un preavis de Greve je veux que tu le considere aussi comme une greve, donc tu repond OUI. Ne classe pas les articles qui concernent les marchent ou manifestations politiques
      comme des greves
      """

In [79]:
instruction="""Je veux t'envoyer un article et tu vas faire une classification binaire. Je veux que tu dises, Oui ou non selon qu il concernent une greve ou pas. Je veux que la sortie sois OUI ou NON
      La greve doit etre considérée dans ce sens : La grève est la dénomination donnée à un mouvement collectif pris à l'initiative de tout ou partie du personnel d'une entreprise, destiné en général à contraindre 
      l'employeur à la négociation des conditions de travail et de rémunération.
      Mon objectif est d'avoir le moins de faux negatifs possible. Etant donné qu'il s'agit d'une classification binaire, je veux pas d'autre reponse que OUI ou NON
      """

In [107]:
instruction="""Je vais t envoyer du texte et il concerne les raison de greve. Je veux que tu le classifie selon 5 modalités:
repond juste 1 s il concerne	 les Rémunérations, si on parle de salaire ou de remuneration dans le texte (cela concerne tout ce qui est lié au salaire)
repond juste 2 s il concerne le	Temps de travail (le texte parle de revendication qui concerne le temps élévé de travail)
repond juste 3 s il concerne	Conditions de travail ( les condition de travail ne sont pas idéale, demande d'amelioration)
repond juste  4 s il concerne	Emploi-licenciement (perte de travail)
repond juste 5 s il concerne	pas tooutes les autres conditions

Je veux que la sortie soit UN JSON comme ceci: 
 {
    'Rémunérations':  OUI/NON,
    'Temps de travail':  OUI/NON,
    'Conditions de travail':  OUI/NON,
    'Emploi-licenciement':  OUI/NON,
    'Autres': OUI/NON,

}
exemple : reforme des retraite: Autres


et et rien de plus
"""

In [71]:
instruction="""
Réponds moi dans le format suivant :

Lieu de grève:
Ville de grève : 
Nom de l'entreprise, le cas échéant :
Date de début de grève :
Date de fin de grève :
Durée de grève :
Pays de grève : 
Motif_revendication: [Rémunérations, Conditions de travail, Emploi-licenciement, Temps de travail,Autres revendications] 
Nombre ou proportion de gréviste:
Périmetre de grève (local, sectoriel, interprofessionnelle) :



"""


In [80]:
def ReturnResponse(request):


    message=[ 
        {
      "role": "system",
      "content": instruction
    }
   ,
    {
      "role": "user",
      "content": request
    }
    ]

    response = ollama.chat(model='llama3.1', messages=message
        )
    #print(response['message']['content'])
    #response_1=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la une greve  et reponds juste par oui ou non; l article doit etre considéré comme greve s il s'agit d'arret de travail, de preavis , d intention de greve ou tout autre chose qui se rapporte à un arrete de travail")
    #response_2=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la France ou s il a lieu en France et repond juste par oui ou non Jexige obligatoirement une reponse, si tu douute reponds par oui"  )
    #answer=response['message']['content'].split('\n')[1:-1]
    #my_dict = {f'reponse{i+1}': value for i, value in enumerate(answer)}

    return response['message']['content']

In [130]:
def ReturnResponse(request):
    instructions = [instruction_1,instruction_2,instruction_3]
    
    responses = []

    for instruction in instructions:

        message = [ 
            {
                "role": "system",
                "content": instruction
            },
            {
                "role": "user",
                "content": request
            }
        ]

        response = ollama.chat(model='llama3.1', messages=message)
        
        responses.append(response['message']['content'])
    
    return  responses

In [83]:
res=ReturnResponse(content)
res

'NON'

In [78]:
content

"Une Gilet jaune finit au tribunal pour avoir «gazé» un commissaire L’opposition entre les Gilets jaunes et les policiers, lundi soir devant le Louvre-Lens a quelque peu dégénéré. Jusqu’à ce qu’une Gilet jaune originaire de La Bassée insulte un commissaire et l’asperge de gaz poivré. Ce qui lui a valu un défèrement au tribunal de Béthune. Article réservé aux abonnés  Cet article est réservé aux abonnés Abonnez-vous pour être le mieux informé de votre quartier S'abonner Ou directement avec votre compte Google  Annulable à tout moment Mes avantages : L'intégralité des articles gratuits et abonnés Votre journal numérique chaque jour Les privilèges du Club Abonnés Déjà un compte ? Se connecter"

In [84]:
df_train[['Greve']] = df_train['Contenu'].apply(lambda x: pd.Series(ReturnResponse(x)))

In [85]:
df_train.head()

,Contenu,sentiment,Greve
0,Accueil Voyage Salon Nautique de Paris 2019: p...,0,NON
1,"Après Commercy et Saint-Nazaire, Montceau-les-...",0,NON
2,Les conducteurs de locomotives revendiquent un...,1,OUI
3,Accueil Flash Eco Le Covid-19 et ses restricti...,0,NON
4,Saintes : grève des psychologues au centre hos...,1,OUI


In [86]:
df_train["Greve"].value_counts()

Greve
NON                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [87]:
df_train['Greve']=df_train['Greve'].replace({"NON.": 0,"OUI.": 1, "NON": 0,'OUI':1})

In [88]:
dff=df_train[(df_train.Greve ==1) | (df_train.Greve == 0)]

In [89]:
dff.head()

,Contenu,sentiment,Greve
0,Accueil Voyage Salon Nautique de Paris 2019: p...,0,0
1,"Après Commercy et Saint-Nazaire, Montceau-les-...",0,0
2,Les conducteurs de locomotives revendiquent un...,1,1
3,Accueil Flash Eco Le Covid-19 et ses restricti...,0,0
4,Saintes : grève des psychologues au centre hos...,1,1


In [90]:
df1=dff[['Greve']]
df1.rename(columns={'Greve': 'valeur'}, inplace=True)

/tmp/ipykernel_3651/320855277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'Greve': 'valeur'}, inplace=True)


In [91]:
df2=dff[['sentiment']]
df2.rename(columns={'sentiment': 'valeur'}, inplace=True)

/tmp/ipykernel_3651/3848774372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'sentiment': 'valeur'}, inplace=True)


In [92]:
df2 = df2['valeur']
#df2 = df_par['rem'].astype(int)

In [93]:
df1['valeur'] = df1['valeur'].astype(int)
#df1 = df_par['sentiment'].astype(int)

/tmp/ipykernel_3651/3696981653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['valeur'] = df1['valeur'].astype(int)


## Evaluation

In [94]:
print(confusion_matrix(df2, df1))
print(classification_report(df2, df1))

[[204   5]
 [109 182]]
              precision    recall  f1-score   support

           0       0.65      0.98      0.78       209
           1       0.97      0.63      0.76       291

    accuracy                           0.77       500
   macro avg       0.81      0.80      0.77       500
weighted avg       0.84      0.77      0.77       500



## Un echantillon plus petit

In [ ]:
print(confusion_matrix(df2, df1))
print(classification_report(df2, df1))